The code in here is somewhat involved mostly because it needs to parallelize the data reading process, otherwise it could be a lot simpler. 

parseSrr is probably what u need to read the most

In [1]:
import pandas as pd
import numpy as np
import re
import os
import math
from multiprocessing import Pool

## init
mySpecie='Homo_sapiens'
outMergedDir='/cellar/users/btsui/all_seq_snp/'+mySpecie+'_all_merged_snp.h5'

##change this dir to point to the updated csv
full_meta_dir="/cellar/users/btsui/Project/METAMAP/notebook/Parsing/sra_dump.csv"
inSrrDir='/nrnb/users/btsui/Data/all_seq/snp/'
tmp_dir='/nrnb/users/btsui/Data/all_seq/tmp/'

In [2]:
#!df -h /data/cellardata/

In [3]:
%%time
#%matplotlib inline
##machine: 5-1



full_meta_df=pd.read_csv(full_meta_dir)

#inSrrDir='/cellar/users/btsui/Project/METAMAP/notebook/RapMapTest/XGS_WGS/'
#existingMergedDf=pd.read_pickle(outMergedDir)

mySpecieDf=full_meta_df[full_meta_df['ScientificName']==mySpecie]

#find the chromosomes 
tmpBedDf=pd.read_csv('/data/cellardata/users/btsui/dbsnp/snp_beds/'+mySpecie+'.bed',header=None,sep='\t')
unique_chroms=tmpBedDf[0].astype(np.str).unique()

### start merging one by one 
if os.path.exists(tmp_dir):
    os.system('rm -r '+tmp_dir)
os.system('mkdir -p '+tmp_dir)


os.chdir(tmp_dir)
#identify non empty files
os.system('ls -la '+inSrrDir+' > ls_out.txt ')
ls_df=pd.read_csv('ls_out.txt',sep='\s+',header=None,names=np.arange(9)).iloc[1:]
#ls_df=
size_S=ls_df[4]
m4=size_S.astype(np.int)>1000
m5=ls_df[8].str.contains('.txt.snp.gz$')
non_empty_files=ls_df[m4&m5][8].str.split('/').str[-1].str.split('.').str[0].values



CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.7 µs


/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,6,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print ('# of files to merge:',len(non_empty_files))

# of files to merge: 253005


In [6]:
%%time
"""
given: srr id 
return: the merged file
"""
def parseSrr(inSrr):
    #print inSrr
    fname=inSrrDir+inSrr+'.txt.snp.gz'
    tmpDf_all=pd.read_csv(fname,sep='\s+',header=None,names=np.arange(50),index_col=None,error_bad_lines=False)
    myCols=['Chr','Pos','Ref','rd_all','','A','C','G','T','N']
    tmpDf=tmpDf_all.iloc[:,:len(myCols)]
    tmpDf.columns=myCols
    tmpDf2=tmpDf.set_index(['Chr','Pos'])
    myBases=['A','C','G','T']
    myL=[]
    for base in myBases:
            splitL=tmpDf2[base].str.split(':',expand=True)
            ### extract  the read count and base quality
            tmpDf5=splitL[[1,3]].astype(np.float)
            tmpDf5.columns=['ReadDepth','AverageBaseQuality']
            myL.append(tmpDf5)
    tmpDf6=pd.concat(myL,keys=myBases,axis=0,names=['base'])
    tmpDf6.columns.name='features'
    mergedDf=tmpDf6.astype(np.uint16)
    non_zero_df=mergedDf[mergedDf['ReadDepth']>0]
    tmpDf7=non_zero_df.reset_index()
    Run_digits=re.search('[DES]RR(\d+)', inSrr)
    Run_Db=re.search('([DES]RR)\d+', inSrr)
    tmpDf7['Run_digits']=Run_digits.group(1)
    tmpDf7['Run_db']=Run_Db.group(1)
    ###convert the datatypes
    tmpDf7['Pos']=tmpDf7['Pos'].astype(np.uint32)    
    tmpDf7['Run_digits']=tmpDf7['Run_digits'].astype(np.uint64)
    tmpDf7['Chr']=tmpDf7['Chr'].astype(np.str).astype('category',
                                                      categories=unique_chroms,ordered=True)
    tmpDf7['Run_db']=tmpDf7['Run_db'].astype(np.str).astype('category',
                                                            categories=['DRR','ERR','SRR'],ordered=True)
    tmpDf7['base']=tmpDf7['base'].astype('category',
                                         categories=myBases,ordered=True)
    srr_pickle_df=tmpDf7.set_index(['Run_db','Run_digits',u'Chr', u'Pos',u'base']).sort_index()
    return srr_pickle_df

### identify files to be merged
fnames=pd.Series(os.listdir(inSrrDir))
snpFnames=fnames[fnames.str.contains('.snp.gz$')]
srrsWithData=snpFnames.str.split('.').str[0]
#mergedSrrs=existingMergedDf.index.get_level_values('Run')
#m1=~srrsWithData.isin(mergedSrrs)
m2=srrsWithData.isin(mySpecieDf['Run'])
m3=srrsWithData.isin(non_empty_files)
toMergeSrrs=srrsWithData[m2&m3].values


CPU times: user 2.86 s, sys: 540 ms, total: 3.4 s
Wall time: 3.93 s


In [8]:
TEST=True
if TEST:
    toRunSrrs=toMergeSrrs[:10]
    chunkSize=5
    nThread=1
else:
    toRunSrrs=toMergeSrrs
    chunkSize=1000
    nThread=64
#optional: free up the memory
if not TEST:
    del mySpecieDf, full_meta_df

def mergeSrrsL(i):
    tmpL=[]
    failedSrrsL=[]
    for srr in toRunSrrs[i:(i+chunkSize)]:
        try:
            tmpL.append(parseSrr(srr))
        except :
            print ('failed: '+srr)
            failedSrrsL.append(srr)
    tmpMergedDf=pd.concat(tmpL)
    #tmpMergedDf=pd.concat([parseSrr(srr) for srr in toRunSrrs[i:(i+chunkSize)]])
    reorderedDf=tmpMergedDf.sort_index()
    reorderedDf.to_pickle(tmp_dir+str(i)+'.pickle.gz',compression='gzip')
    return failedSrrsL

Chunks=np.arange(0, len(toRunSrrs),chunkSize)
if TEST:
    failed_srr_l=map(mergeSrrsL,Chunks.tolist())
else:
    from multiprocessing import Pool
    p=Pool(nThread)
    ### sweep for uncompleted chunks
    failed_srr_l=p.map(mergeSrrsL,Chunks.tolist())
    p.close()




In [9]:
#!ls /cellar/users/btsui/Data/dbsnp/

In [10]:
snpBed='/cellar/users/btsui/Data/dbsnp/snp_beds/'+mySpecie+'.bed'


### find the data that overlap exactly
tmpDf_ref_all_snps_df=pd.read_csv(snpBed,
                      sep='\s+',header=None,names=np.arange(50),index_col=None,error_bad_lines=False)
myCols=['Chr','Pos','Ref','rd_all','','A','C','G','T','N']
tmpDf=tmpDf_ref_all_snps_df.iloc[:,:len(myCols)]
tmpDf.columns=myCols
myReindexedDf=tmpDf.set_index(['Chr','Pos'])
refIndex=myReindexedDf.index

/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### merging the files

In [ ]:
"""
changes: need to do the split for different bins in here

Map the regions into the right bin
"""
#myInDir=''
#feature=''
from multiprocessing import Pool
import pandas as pd
import numpy as np
import os
tmp_dir='/nrnb/users/btsui/Data/all_seq/tmp/'
TEST=False

"""
change directory and identify intermediate pickle objects
"""
os.chdir(tmp_dir)
my_ls_S=pd.Series(os.listdir('./'))
ChunksFnames=my_ls_S[my_ls_S.str.contains('.pickle.gz$')].values

### create another dirctory for 
tmp_dir_2='/nrnb/users/btsui/Data/all_seq/tmp_2/'
os.system('rm -r '+tmp_dir_2)
os.system('mkdir '+tmp_dir_2)

"""
input: pickle name
output: split into chunks 
"""
feature_for_hash='Pos'
#2483 blocks per file 
def splitChunk(inChunkFname):
    tmpDf3=pd.read_pickle(inChunkFname).reset_index()
    my_base_shift=int(10**5)#int(10**(max_size_order-n))
    my_block_S=(tmpDf3[feature_for_hash]/my_base_shift).astype(np.int)*my_base_shift
    ##add one for the loop at the end
    tmpDf3['block']=my_block_S #6.5s
    tmpDf3=tmpDf3.sort_values(feature_for_hash) # might improve perfromance of group by
    """
    export chunks
    """
    g=tmpDf3.groupby('block')
    for myBlock,tmpDf9 in g:
        myKeyName='Chunk_'+str(inChunkFname.split('.')[0])+'.'+feature_for_hash+'_block_'+str(myBlock)
        #print (tmp_dir_2+myKeyName)
        tmpDf9.to_pickle(tmp_dir_2+myKeyName)
        
#splitChunk('60700.pickle.gz')

"""
test on a small number of files first 
"""
p=Pool(64)
if TEST:
    p.map(splitChunk,ChunksFnames[:10])
else:
    p.map(splitChunk,ChunksFnames)
p.close()


"""

"""
tmp_dir_3='/nrnb/users/btsui/Data/all_seq/tmp_3/'
os.system('rm -r '+tmp_dir_3)
os.system('mkdir '+tmp_dir_3)
my_chunked_pickles=pd.Series(os.listdir(tmp_dir_2))
tmpDf6=pd.DataFrame({"chunk":my_chunked_pickles.str.split('.').str[1],"fname":my_chunked_pickles})
g=tmpDf6.groupby('chunk')['fname']
myChunks=g.groups.keys()

def mergeChunks(inputChunk):
    myFiles=g.get_group(inputChunk)
    myL=[]
    for myFile in myFiles:
        myL.append(pd.read_pickle(tmp_dir_2+myFile))
    myMergedDf=pd.concat(myL,axis=0)
    myMergedDf.set_index(['Chr','base','Run_db']).to_hdf(tmp_dir_3+inputChunk,'chunk',mode='w',format='fixed')

p=Pool(64)
p.map(mergeChunks,myChunks)
p.close()
"""
for each chunks
"""

In [ ]:
"""
for each chunk, merge into a hdf5 file 
merge all the tiny chunks into 
"""

In [23]:
myDfDf=pd.read_hdf('Pos_block_140700000',mode='r')

In [32]:
#subDf=myDfDf.loc['7']
#subDf[subDf['Pos']==140753336].reset_index()#.index.get_level_values('base')

In [15]:
"""
changes: need to do the split for different bins in here

Map the regions into the right bin
"""
#myInDir=''
#feature=''
import pandas as pd
from multiprocessing import Pool
import numpy as np
import os
"""
change directory and identify intermediate pickle objects
"""
os.chdir(tmp_dir)
my_ls_S=pd.Series(os.listdir('./'))
ChunksFnames=my_ls_S[my_ls_S.str.contains('.pickle.gz$')].values


### create another dirctory for 
tmp_dir_2='/nrnb/users/btsui/Data/all_seq/tmp_2/'
os.system('rm -r '+tmp_dir_2)
os.system('mkdir '+tmp_dir_2)

"""
input: pickle name
output: split into chunks 
"""
feature_for_hash='Pos'
inChunkFname=ChunksFnames[0]
tmpDf3=pd.read_pickle(inChunkFname).reset_index()
my_max=tmpDf3[feature_for_hash].max()

print (my_max)# 6121752
#max_size_order=math.ceil(np.log10(my_max))
#print ('order of max',max_size_order)
#n=4
my_base_shift=int(10**5)#int(10**(max_size_order-n))
my_block_S=(tmpDf3[feature_for_hash]/my_base_shift).astype(np.int)*my_base_shift

nBlocks=my_block_S.max()
print ('# of blocks',)
blocks_VC=my_block_S.value_counts()
##add one for the loop at the end
my_range=blocks_VC.index#np.arange(0,nBlocks+1,my_base_shift)
print ('range size: ',len(my_range))
outH5Name=outMergedDir.replace('.h5','.'+feature_for_hash+'.'+str(my_base_shift)+'.chunked.h5')
print ('output name:',outH5Name)
os.system('rm '+outH5Name)
%time tmpDf3['block']=my_block_S #6.5s
%time tmpDf3=tmpDf3.sort_values(feature_for_hash) # 30 mins

"""
export chunks
"""
#myOrderDict={'Pos':[u'Chr', u'Pos',u'base','Run_digits','Run_db'],
#            'Run_digits':['Run_digits','Run_db',u'Chr', u'Pos',u'base']}
g=tmpDf3.groupby('block')
for myBlock,tmpDf9 in g:
    tmpDf9=tmpDf3[tmpDf3['block']==myBlock]# For run, 27s, For Pos: 2 min :30 s
    my_index_order=myOrderDict[feature_for_hash]
    #tmpDf10=tmpDf9.set_index(my_index_order) 
    myKeyName=feature_for_hash+'_'+str(myBlock)+'.block_'+str(myBlock)
    print (tmp_dir_2+myKeyName)
    tmpDf9.to_pickle(tmp_dir_2+myKeyName)
    #print (myKeyName,tmpDf9.shape)
    #myKeyName


array(['tmp.pickle.gz', '0.pickle.gz', '1000.pickle.gz', 'tmp2.pickle.gz'], dtype=object)

In [ ]:
### merge the files
"""
my_max=tmpDf3[feature_for_hash].max()

print (my_max)# 6121752
max_size_order=math.ceil(np.log10(my_max))
print ('order of max',max_size_order)
n=4
my_base_shift=int(10**(max_size_order-n))
my_block_S=(tmpDf3[feature_for_hash]/my_base_shift).astype(np.int)*my_base_shift
##check the data type of my_block_S and Run_digits
#my_block_S=my_block_S
### make the data become int 
nBlocks=my_block_S.max()
print ('# of blocks',)

blocks_VC=my_block_S.value_counts()

##add one for the loop at the end
my_range=blocks_VC.index#np.arange(0,nBlocks+1,my_base_shift)
print ('range size: ',len(my_range))
outH5Name=outMergedDir.replace('.h5','.'+feature_for_hash+'.'+str(my_base_shift)+'.chunked.h5')
print ('output name:',outH5Name)
os.system('rm '+outH5Name)
%time tmpDf3['block']=my_block_S #6.5s
%time tmpDf3=tmpDf3.sort_values(feature_for_hash) # 30 mins

"""
"""
changes: need to do the split for different bins in here


"""
"""
myTmpL=[]
my_ls_S=pd.Series(os.listdir('./'))
ChunksFnames=my_ls_S[my_ls_S.str.contains('.pickle.gz$')].values
for chunkFname in ChunksFnames:
    tmpDf=pd.read_pickle(chunkFname)
    tmpDf2=tmpDf.reset_index()
    #tmpDf3=tmpDf2.set_index(['Chr','Pos'])
    #m=tmpDf3.index.isin(refIndex)
    subTmpDf2=tmpDf2#[m]
    print '% sites in targets:', (m.mean())
    print subTmpDf2.shape
    myTmpL.append(subTmpDf2)
    
mergedDf=pd.concat(myTmpL)

#mergedS

all_mergedDf=mergedDf.set_index(
    ['Run_db','Run_digits']).sort_index()
#all_mergedDf.to_pickle(outMergedDir)
"""
all_mergedDf.to_hdf(outMergedDir,key='master',mode="w")
allFailedSrrs=pd.Series(reduce(lambda a,x:a+x,failed_srr_l,[]))
allFailedSrrs.to_csv('/nrnb/users/btsui/Data/all_seq/'+mySpecie+'.failed.srrs.txt')


In [ ]:
###change corrdinate to chromsome base for slicing
#mySpecie='Homo_sapiens'
#skymap_snp_dir='/cellar/users/btsui/all_seq_snp/'+mySpecie+'_all_merged_snp.h5'
sorted_snp_dir='/cellar/users/btsui/all_seq_snp/'+mySpecie+'_all_merged_snp.chrom_pos_sorted.h5'
tmpDf=pd.read_hdf(outMergedDir,key='master',mode='r')
tmpDf4=tmpDf.reset_index()
sortedDf=tmpDf4.set_index( [u'Chr', u'Pos',u'base']).sort_index()
sortedDf.to_hdf(sorted_snp_dir,key='master',mode='w')

### scratch

In [ ]:
#sorted_snp_dir='/cellar/users/btsui/all_seq_snp/'+mySpecie+'_all_merged_snp.chrom_pos_sorted.h5'



tmpDf=pd.read_hdf(outMergedDir,key='master',mode='r')
tmpDf3=tmpDf.reset_index()
tmpDf3['base']=tmpDf3['base'].astype('category')

tmpDf3['Chr']=tmpDf3['Chr'].astype('category')
tmpDf3['Run_db']=tmpDf3['Run_db'].astype('category')

tmpDf3['Pos']=tmpDf3['Pos'].astype(np.uint32)
tmpDf3['Run_digits']=tmpDf3['Run_digits'].astype(np.uint32)

#tmpDf4=tmpDf3.set_index(['Run_digits','Run_db',u'Chr', u'Pos',u'base'])
#tmpDf5=tmpDf4.sort_index()
outRunSortedOutDir=outMergedDir.replace('.h5','.pickle')
os.system('rm '+outRunSortedOutDir)
tmpDf3.to_pickle(outRunSortedOutDir)
#tmpDf3.to_hdf(outRunSortedOutDir,key='master',mode='w')

### test exporting to hdf5 data

In [ ]:
'/cellar/users/btsui/'

In [ ]:
import pandas as pd
import numpy as np
import re
import os
from multiprocessing import Pool

## init
mySpecie='Homo_sapiens'
outMergedDir='/cellar/users/btsui/all_seq_snp/'+mySpecie+'_all_merged_snp.h5'


%time tmpDf=pd.read_hdf(outMergedDir,key='master',mode='r')
%time tmpDf3=tmpDf.reset_index()
"""
CPU times: user 8.21 s, sys: 19.9 s, total: 28.2 s
Wall time: 1min 18s
CPU times: user 1min 35s, sys: 57.8 s, total: 2min 33s
Wall time: 2min 33s

"""
%time tmpDf5=tmpDf3.set_index(['Run_digits','Run_db'])
"""
#CPU times: user 2min 7s, sys: 1min 54s, total: 4min 1s
Wall time: 4min 1s
"""
%time tmpDf5.to_hdf('./test_wo_chroms.hdf','master',mode='w',append=False,format='table')
#Exception: cannot find the correct atom type -> [dtype->object,items->Index(['Chr', 'base'], dtype='object')] 


In [36]:
math.ceil()

In [ ]:
#myCol=np.array(['Run_digits','Run_db',u'Chr', u'Pos',u'base'])
### what's the datatype for those guys:


In [ ]:
### let's just 
import pandas as pd
import numpy as np
import re
import os
import math

from multiprocessing import Pool

## init
mySpecie='Homo_sapiens'
outMergedDir='/cellar/users/btsui/all_seq_snp/'+mySpecie+'_all_merged_snp.h5'

%time tmpDf=pd.read_hdf(outMergedDir,key='master',mode='r')
%time tmpDf3=tmpDf.reset_index()

#feature_for_hash='Run_digits'
feature_for_hash='Pos'
myOrderDict={'Pos':[u'Chr', u'Pos',u'base','Run_digits','Run_db'],
            'Run_digits':['Run_digits','Run_db',u'Chr', u'Pos',u'base']}
my_max=tmpDf3[feature_for_hash].max()

print (my_max)# 6121752
max_size_order=math.ceil(np.log10(my_max))
print ('order of max',max_size_order)
n=4
my_base_shift=int(10**(max_size_order-n))
my_block_S=(tmpDf3[feature_for_hash]/my_base_shift).astype(np.int)*my_base_shift
##check the data type of my_block_S and Run_digits
#my_block_S=my_block_S
### make the data become int 
nBlocks=my_block_S.max()
print ('# of blocks',)

blocks_VC=my_block_S.value_counts()

##add one for the loop at the end
my_range=blocks_VC.index#np.arange(0,nBlocks+1,my_base_shift)
print ('range size: ',len(my_range))
outH5Name=outMergedDir.replace('.h5','.'+feature_for_hash+'.'+str(my_base_shift)+'.chunked.h5')
print ('output name:',outH5Name)
os.system('rm '+outH5Name)
%time tmpDf3['block']=my_block_S #6.5s
%time tmpDf3=tmpDf3.sort_values(feature_for_hash) # 30 mins
#%time g=tmpDf3.groupby('block') #1min 35s
#
for myBlock in my_range:
#for myBlock,tmpDf9 in g:
    """
    in Pos, 
    """
    %time tmpDf9=tmpDf3[tmpDf3['block']==myBlock]# For run, 27s, For Pos: 2 min :30 s
    
    my_index_order=myOrderDict[feature_for_hash]
    
    #tmpDf10=tmpDf9.set_index(my_index_order) 
    myKeyName=feature_for_hash+'_'+str(myBlock)
    print (myKeyName,tmpDf9.shape)
    
    tmpDf9.to_hdf(outH5Name,myKeyName,mode='a',format='fixed')
    #if myBlock >3000:
    #    break

In [4]:
#('range size: ', 1201)

0.0015764881876101904

In [3]:
### make sure the results is chunked as expects
#!ls /cellar/users/btsui/all_seq_snp/
#2 seconds per data push 

Acinetobacter_baumannii_all_merged_snp.pickle
Acropora_millepora_all_merged_snp.pickle
activated_sludge_metagenome_all_merged_snp.pickle
Aedes_aegypti_all_merged_snp.pickle
air_metagenome_all_merged_snp.pickle
algae_metagenome_all_merged_snp.pickle
anaerobic_digester_metagenome_all_merged_snp.pickle
Anopheles_arabiensis_all_merged_snp.pickle
Anopheles_gambiae_all_merged_snp.pickle
Apis_mellifera_all_merged_snp.pickle
aquatic_metagenome_all_merged_snp.pickle
Arabidopsis_thaliana_all_merged_snp.pickle
Bacteria_all_merged_snp.pickle
biofilm_metagenome_all_merged_snp.pickle
bioreactor_metagenome_all_merged_snp.pickle
bird_metagenome_all_merged_snp.pickle
Boechera_stricta_all_merged_snp.pickle
Bordetella_pertussis_all_merged_snp.pickle
Bos_taurus_all_merged_snp.pickle
bovine_gut_metagenome_all_merged_snp.pickle
bovine_metagenome_all_merged_snp.pickle
Brachypodium_distachyon_all_merged_snp.pickle
Brassica_napus_all_merged_snp.pickle
Brassica_rapa_all_merged_snp.pickle
Burkholderia_pseudomall

In [ ]:
#7 digits, 
# 18gb to 0.1
#create an hdf5 object, 
tmpDf.to_hdf('./test.h5','master',mode='w',append=False,format='table')

In [ ]:
%time tmpDf.to_hdf('./test.h5','master',mode='w',append=False,format='table')


testDf=tmpDf.head()

testDf.to_hdf('./test.h5','master',mode='w',format='table')
#export succceedded, but the slicing operation time takes forever, the path only lead from /master/ to py table. 


In [1]:
import pandas as pd
import numpy as np

In [2]:
#v6
#14.9s 
%time testDf20=pd.read_hdf('/cellar/users/btsui/test.pos.chunked.h5','Run_digits_790000',mode='r')

CPU times: user 3.34 s, sys: 11.4 s, total: 14.7 s
Wall time: 14.9 s


In [9]:
np.log10(testDf20.shape[0])

7.7110316966216308

(51408117, 7)

In [ ]:
#dataset_name   = '/Configure:0000/Run:0000/CalibCycle:0000/Camera::FrameV1/XppSb4Pim.1:Tm6740.1/image'


In [10]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

/cellar/users/btsui/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
hdf5_file_name='/cellar/users/btsui/test.h5'

In [12]:
f  = h5py.File(hdf5_file_name, 'r') 

In [23]:
table=f['/master/table']
#obj.iteritems()

In [ ]:
table[1000:2000]

In [33]:
table.dtype

dtype([('index', '<i8'), ('values_block_0', '<u2', (2,)), ('base', 'S1'), ('Pos', '<i8'), ('Chr', 'S2'), ('Run_digits', '<i8'), ('Run_db', 'S3')])

In [26]:
table.shape

(1456652195,)

In [19]:
for i,myObj in enumerate(obj.iteritems()):
    print myObj
    if i>10:
        break
    

(u'_i_table', None)
(u'table', <HDF5 dataset "table": shape (1456652195,), type "|V34">)


In [35]:
%time posDf=pd.read_hdf('/cellar/users/btsui/test.h5','master',where='Run_digits=15999')


ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/cellar/users/btsui/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1118, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/cellar/users/btsui/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 300, in wrapped
    return f(*args, **kwargs)
  File "/cellar/users/btsui/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 345, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/cellar/users/btsui/anaconda2/lib/python2.7/inspect.py", line 1051, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/cellar/users/btsui/anaconda2/lib/python2.7/inspect.py", line 1011, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/cellar/users/btsui/anaconda2/lib/python2.7/inspect.py", line 450, in getsourcefile
    if os.path.exists(f

IndexError: string index out of range

In [2]:
"""
the multindex doesn't work no matter what
just use pickle for now: 
"""

In [ ]:

"""
[u'Chr', u'Pos',u'base','Run']
-rw-r--r-- 1 btsui users 1.8M Jan  2 16:42 0.pickle.gz
-rw-r--r-- 1 btsui users 2.0M Jan  2 16:42 10.pickle.gz

['Run', u'Chr', u'Pos',u'base']
-rw-r--r-- 1 btsui users 1.9M Jan  2 16:43 0.pickle.gz
-rw-r--r-- 1 btsui users 2.2M Jan  2 16:44 10.pickle.gz
"""

In [ ]:
"""
### where did 1000.pickle.gz go from the first iteration
### out of 82, only 60 processed, what about the rest?
### 
"""

In [ ]:
#!rm /tmp/btsui/snp_merged/0.pickle

In [15]:
#18G
myDf=pd.read_hdf('/cellar/users/btsui/all_seq_snp/Homo_sapiens_all_merged_snp.h5',key='master')

In [17]:
#myDf

In [ ]:
### keep everything in the same place?
##

In [ ]:
### read in all the human.
###take only those in human and merge all of them together, be only additivite, add only those where it has not been added. 
#srr_pickle_df.iloc[:0].to_pickle('/data/cellardata/users/btsui/dbsnp/empty_base_snp.pickle')

In [ ]:
## merge all the data in bulk, 
### generate an empty pickle for each species?

In [ ]:
#orignal pickle, 2.7m
#!ls -lah ./tmp.pickle

In [ ]:
#!ls -lah tmp2.pickle.gz

In [ ]:
#!rm ./tmp2.pickle

In [ ]:
### merge all the files, by 1000 steps

In [ ]:
srr_pickle_df.to_pickle('./tmp2.pickle')
#srr_pickle_df.to_pickle('./tmp.pickle')

In [ ]:
!ls -lah ./tmp.pickle  ./tmp2.pickle

In [ ]:
!rm ./tmp2.pickle.gz
!rm ./tmp.pickle.gz

In [ ]:

!gzip ./tmp2.pickle

In [ ]:

!gzip ./tmp.pickle

In [ ]:
!ls -lah tmp2.pickle.gz

In [ ]:
!ls -lah tmp.pickle.gz

In [ ]:
### merge the pickle 

In [ ]:
#0.4 mb
np.log10(0.4*(4*(10**6)))

In [ ]:
### the multindex doesn't increase the 

In [ ]:

#no filtering on variants
countDf=tmpDf6[tmpDf6.rd>0]#.rd.value_counts()
##at a position, tell what's the value


In [ ]:
subDf.head()

In [ ]:
my_data_L=[]
my_key_L=[]
g=countDf.stack().groupby(['base','features'])
for myTuple, subS in g:
    sparse_Ids=subS.index.get_level_values(index_name)
    tmp_row_col=(sparse_Ids,np.zeros(len(sparse_Ids)))
    tmp_array=sparse.csc_matrix((subS ,tmp_row_col),shape=tmp_shape)
    my_data_L.append(tmp_array)
    my_key_L.append(myTuple)

In [ ]:
sparseM=sparse.hstack(my_data_L)

In [ ]:
index_name='Chr_Pos_Id'
baseDf.groupby(index_name)
#csc_matrix((data, (row, col)), shape=(3, 3)).toarray()
attrib='rd'
tmp_shape=(Chr_Pos_to_ID_S.max(),1)
for my_base, subDf in countDf.groupby(level='base'):
    sparse_Ids=subDf.index.get_level_values(index_name)
    
    tmp_row_col=(sparse_Ids,np.zeros(len(sparse_Ids)))
    
    tmpS=sparse.csc_matrix((subDf[attrib] ,tmp_row_col),shape=tmp_shape)

In [ ]:
countDf.shape

In [ ]:
#countDf

In [289]:
#full_meta_df['Run']

In [282]:
import pandas as pd
import numpy as np
import re
import os
from multiprocessing import Pool

## init
mySpecie='Homo_sapiens'
outMergedDir='/cellar/users/btsui/all_seq_snp/'+mySpecie+'_all_merged_snp.h5'

full_meta_dir="/cellar/users/btsui/Project/METAMAP/notebook/Parsing/sra_dump.csv"
full_meta_df=pd.read_csv(full_meta_dir)

#inSrrDir='/cellar/users/btsui/Project/METAMAP/notebook/RapMapTest/XGS_WGS/'
inSrrDir='/nrnb/users/btsui/Data/all_seq/snp/'
existingMergedDf=pd.read_pickle(outMergedDir)

In [292]:
existingMergedDf.groupby(['Run_db','Run_digits']).size()

KeyError: 'Run_db'

In [3]:
import pandas as pd
import numpy as np

In [4]:
 s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])

In [7]:
s.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [6]:
s.index.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [11]:
pd.__path__

['/cellar/users/btsui/anaconda2/lib/python2.7/site-packages/pandas']

In [12]:
store = pd.HDFStore('test.h5','w')


In [ ]:
### check compression ratio with at last ten reads: 

In [ ]:
### select by variants, for select by 